In [ ]:
# Initial development

# Import classes and methods
from pybrors.dicom import DicomFile, DicomDir

# Create file instance
tmp = DicomFile(file_path="../data/dicom/supported_file.dcm")
print(tmp)
print(tmp.file_path)
print(tmp.file_dir)
print(tmp.file_name)
print(tmp.file_ext)

# Create directory instance
tmp = DicomDir(dir_path="../data/dicom/dicom_exam")
print(tmp)
print(tmp.file_list)
